In [1]:
from glob import glob
from collections import Counter
import pandas as pd

from IPython.display import display
import re
import os

In [2]:
DATA_DRIVE_PATH = "G:/AzureBackup/"
NER_FILES={
    "Finin": {
        "train": "/datadrive/Datasets/lowlands-data/LREC2014/twitter_ner/data/finin.train.tsv",
        "test": "/datadrive/Datasets/lowlands-data/LREC2014/twitter_ner/data/finin.test.tsv.utf8",
    },
    "Hege": {
        "test": "/datadrive/Datasets/lowlands-data/LREC2014/twitter_ner/data/hege.test.tsv",
    },
    "Ritter": {
        "lowlands-test": "/datadrive/Datasets/lowlands-data/LREC2014/twitter_ner/data/ritter.test.tsv",
        "train": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/ner.train.txt",
        "dev": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/ner.dev.txt",
        "test": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/ner.test.txt",
    },
    "YODIE": {
        "train": "/datadrive/Datasets/Twitter/YODIE/data/training.conll",
        "test": "/datadrive/Datasets/Twitter/YODIE/data/testing.conll"
    },
    "WNUT_2016": {
        "train": "/datadrive/Codes/multi-task-nlp-keras/data/WNUT_NER/train.tsv",
        "test": "/datadrive/Codes/multi-task-nlp-keras/data/WNUT_NER/test.tsv",
        "dev": "/datadrive/Codes/multi-task-nlp-keras/data/WNUT_NER/dev.tsv",
    },
    "WNUT_2017": {
        "train": "/datadrive/Codes/multi-task-nlp-keras/data/WNUT_2017/wnut17train.conll",
        "dev": "/datadrive/Codes/multi-task-nlp-keras/data/WNUT_2017/emerging.dev.conll",
        "test": "/datadrive/Codes/multi-task-nlp-keras/data/WNUT_2017/emerging.test.annotated",
    },
    "MSM_2013": {
        "train": "/datadrive/Datasets/Twitter/MSM2013/data/msm2013-ce_challenge_gs/TweetsTrainingSetCH.tsv.conll",
        "test": "/datadrive/Datasets/Twitter/MSM2013/data/msm2013-ce_challenge_gs/goldStandard.tsv.conll",
    },
    "NEEL2016": {
        "train": "/datadrive/Datasets/Twitter/microposts-NEEL/processed/2016/microposts2016-neel-training_neel.gs.conll",
        "dev": "/datadrive/Datasets/Twitter/microposts-NEEL/processed/2016/microposts2016-neel-dev_neel.gs.conll",
        "test": "/datadrive/Datasets/Twitter/microposts-NEEL/processed/2016/microposts2016-neel-test_neel.gs.conll",
    },
    "BROAD": {
        "a": "/datadrive/Datasets/Twitter/broad_twitter_corpus/a.conll",
        "b": "/datadrive/Datasets/Twitter/broad_twitter_corpus/b.conll",
        "e": "/datadrive/Datasets/Twitter/broad_twitter_corpus/e.conll",
        "f": "/datadrive/Datasets/Twitter/broad_twitter_corpus/f.conll",
        "g": "/datadrive/Datasets/Twitter/broad_twitter_corpus/g.conll",
        "h": "/datadrive/Datasets/Twitter/broad_twitter_corpus/h.conll",
        
        "split-train": "/datadrive/Datasets/Twitter/broad_twitter_corpus/data_splits/train.conll",
        "split-dev": "/datadrive/Datasets/Twitter/broad_twitter_corpus/data_splits/dev.conll",
        "split-test": "/datadrive/Datasets/Twitter/broad_twitter_corpus/data_splits/test.conll",
        
    },
    "MultiModal": {
        "train": "/datadrive/Datasets/Twitter/NERmultimodal/data/train.conll",
        "dev": "/datadrive/Datasets/Twitter/NERmultimodal/data/dev.conll",
        "test": "/datadrive/Datasets/Twitter/NERmultimodal/data/test.conll",
    }
}


POS_FILES={
    "Owoputi_2013": {
        "train": "/datadrive/Datasets/Twitter/TweeboParser/ark-tweet-nlp-0.3.2/data/twpos-data-v0.3/oct27.splits/oct27.train",
        "traindev": "/datadrive/Datasets/Twitter/TweeboParser/ark-tweet-nlp-0.3.2/data/twpos-data-v0.3/oct27.splits/oct27.traindev",
        "dev": "/datadrive/Datasets/Twitter/TweeboParser/ark-tweet-nlp-0.3.2/data/twpos-data-v0.3/oct27.splits/oct27.dev",
        "test": "/datadrive/Datasets/Twitter/TweeboParser/ark-tweet-nlp-0.3.2/data/twpos-data-v0.3/oct27.splits/oct27.test",
        "daily547": "/datadrive/Datasets/Twitter/TweeboParser/ark-tweet-nlp-0.3.2/data/twpos-data-v0.3/daily547.conll"
    },
    
    #"LexNorm_Li_2015": {
    #    "dev": "/datadrive/Datasets/Twitter/wnut-2017-pos-norm/data/1.dev.gold.noUserWWW",
    #    "test": "/datadrive/Datasets/Twitter/wnut-2017-pos-norm/data/test_L.gold",
    #    "test-Owoputi": "/datadrive/Datasets/Twitter/wnut-2017-pos-norm/data/test_O.gold",
    #    "train": "/datadrive/Datasets/Twitter/wnut-2017-pos-norm/data/train_pos.noUserWWW"
    #},
    
    ## Next 3 use Universal POS mappings: 
    "Foster_2011": {
        "test": "/datadrive/Datasets/lowlands-data/ACL2014/crowdsourced_POS/data/foster-twitter.test",
        "twitie-dev": "/datadrive/Datasets/Twitter/twitter-pos-bootstrap/data/foster_dev.conll",
        "twitie-test": "/datadrive/Datasets/Twitter/twitter-pos-bootstrap/data/foster_eval.conll"
    },
    "Ritter_2011": {
        "lowlands-test": "/datadrive/Datasets/lowlands-data/ACL2014/crowdsourced_POS/data/ritter.test",
        "train": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/pos.cleaned.train.txt",
        "dev": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/pos.cleaned.dev.txt",
        "test": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/pos.cleaned.test.txt",
        # Save as above
        #"twitie-train": "/datadrive/Datasets/Twitter/twitter-pos-bootstrap/data/ritter_train.conll",
        #"twitie-dev": "/datadrive/Datasets/Twitter/twitter-pos-bootstrap/data/ritter_dev.conll",
        #"twitie-test": "/datadrive/Datasets/Twitter/twitter-pos-bootstrap/data/ritter_eval.conll"
    },
    "lowlands": {
        "test": "/datadrive/Datasets/lowlands-data/ACL2014/crowdsourced_POS/data/lowlands.test"
    },
    "Gimple_2012": {
        "test": "/datadrive/Datasets/lowlands-data/ACL2014/crowdsourced_POS/data/gimpel.GOLD"
    },
    "Bootstrap_2013": {
        # Full PTB tagset, plus four custom tags (USR, HT, RT, URL)
        # Skipping because not relevant
        #"train": "/datadrive/Datasets/Twitter/twitter-pos-bootstrap/data/bootstrap.conll"
    },
    "Tweetbankv2": {
        "dev": "/datadrive/Datasets/Twitter/Tweebank/pos/en-ud-tweet-dev.txt",
        "train": "/datadrive/Datasets/Twitter/Tweebank/pos/en-ud-tweet-train.txt",
        "test": "/datadrive/Datasets/Twitter/Tweebank/pos/en-ud-tweet-test.txt",
    }
}

CHUNKING_FILES = {
    "Ritter": {
        "train": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/chunk.train.conll",
        "dev": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/chunk.dev.conll",
        "test": "/datadrive/Datasets/Twitter/RitterNER/twitter_processed/chunk.test.conll",
    }
}

SENTIMENT_FILES={
    "SMILE": {
        "train": "/datadrive/Datasets/Twitter/SMILE/smile-annotations-final.csv",
    },
}

SUPERSENSE_TAGGING_FILES={
    "Ritter": {
        "train": "/datadrive/Datasets/Twitter/supersense-data-twitter/ritter-train.tsv",
        "dev": "/datadrive/Datasets/Twitter/supersense-data-twitter/ritter-dev.tsv",
        "test": "/datadrive/Datasets/Twitter/supersense-data-twitter/ritter-eval.tsv"
    },
    "Johannsen_2014": {
        "test": "/datadrive/Datasets/Twitter/supersense-data-twitter/in-house-eval.tsv"
    }
}

FRAME_SEMANTICS_FILE={
    "Sogaard_2015": {
        "gavin": "/datadrive/Datasets/lowlands-data/AAAI15/conll/all.gavin",
        "maria": "/datadrive/Datasets/lowlands-data/AAAI15/conll/all.maria",
        "sara": "/datadrive/Datasets/lowlands-data/AAAI15/conll/all.sara"
    }
}

DIMSUM_FILES = {
    # Following data is already part of dimsum
    #"Lowlands": {
    #    "test": "/datadrive/Datasets/Twitter/dimsum-data/conversion/original/lowlands.UPOS2.tsv"
    #},
    #"Ritter": {
    #    "test": "/datadrive/Datasets/Twitter/dimsum-data/conversion/original/ritter.UPOS2.tsv"
    #},
    #"Streusle": {
    #    "test": "/datadrive/Datasets/Twitter/dimsum-data/conversion/original/streusle.upos.tags"
    #}, 
    "DiMSUM_2016": {
        # Made in combination with ritter, streusle, lowlands
        # 55579 ewtb
        # 3062 lowlands
        # 15185 ritter
        "train": "/datadrive/Datasets/Twitter/dimsum-data/conll/dimsum16.train",
        # 3516 ted
        # 6357 trustpilot
        # 6627 tweebank
        "test": "/datadrive/Datasets/Twitter/dimsum-data/conll/dimsum16.test"
    }
}

PARSING_FILES={
    "Kong_2014": {
        "train": "/datadrive/Datasets/Twitter/TweeboParser/Tweebank/Train_Test_Splited/train",
        "test": "/datadrive/Datasets/Twitter/TweeboParser/Tweebank/Train_Test_Splited/test",
    }
}

WEB_TREEBANK={
    "DenoisedWebTreebank": {
        "dev": "/datadrive/Datasets/Twitter/DenoisedWebTreebank/data/DenoisedWebTreebank/dev.conll",
        "test": "/datadrive/Datasets/Twitter/DenoisedWebTreebank/data/DenoisedWebTreebank/test.conll"
    }
}

NORMALIZED={
    "DenoisedWebTreebank": {
        "dev": "/datadrive/Datasets/Twitter/DenoisedWebTreebank/data/DenoisedWebTreebank/dev.normalized",
        "test": "/datadrive/Datasets/Twitter/DenoisedWebTreebank/data/DenoisedWebTreebank/test.normalized"
    }
}

PARAPHRASE_SEMANTIC_FILES={
    "SemEval-2015 Task 1": {
        #  Topic_Id | Topic_Name | Sent_1 | Sent_2 | Label | Sent_1_tag | Sent_2_tag |
        # Map labels as follows
        # paraphrases: (3, 2) (4, 1) (5, 0)
        # non-paraphrases: (1, 4) (0, 5)
        # debatable: (2, 3)  which you may discard if training binary classifier
        
        "train": "/datadrive/Datasets/Twitter/SemEval-PIT2015-github/data/train.data",
        "dev": "/datadrive/Datasets/Twitter/SemEval-PIT2015-github/data/dev.data",
        "test": "/datadrive/Datasets/Twitter/SemEval-PIT2015-github/data/test.data"
    }
}

In [3]:
SEQ_SPLITTER = re.compile(r'\n\s*\n', flags=re.M)
SEQ_SPLITTER.split("""shubh POS
mish pos
she pos

shubh POS
mish pos
she pos
    
shubh POS
mish pos
she pos

""")

['shubh POS\nmish pos\nshe pos',
 'shubh POS\nmish pos\nshe pos',
 'shubh POS\nmish pos\nshe pos',
 '']

In [4]:
def read_conll_data(filename, ncols=2):
    with open(filename, encoding='utf-8') as fp:
        for seq in SEQ_SPLITTER.split(fp.read()):
            seq_ = []
            for line in seq.splitlines():
                line = line.rstrip()
                if not line:
                    continue
                values = line.split("\t")
                if len(values) < ncols:
                    # Skip invalid lines
                    continue
                seq_.append(values)
            if not seq_:
                seq_ = []
                continue
            yield seq_
                

In [5]:
def get_ner_label(label, idx=1):
    label = label.strip().upper()
    if label == "O":
        return label
    if idx is None:
        return label
    return label.split('-', 1)[idx].strip()

def get_simple_label(label):
    if label:
        return label
    return "O"

def get_file_stats(
    filename,
    label_processor=None,
    label_col_id=-1,
    skip_other=True,
    ncols=2
):
    if label_processor is None:
        label_processor = lambda x: x
    total_seq = 0
    total_tokens = 0
    token_types = Counter()
    for i, seq in enumerate(read_conll_data(filename, ncols=ncols)):
        total_seq += 1
        total_tokens += len(seq)
        try:
            for item in seq:
                label = label_processor(item[label_col_id])
                if skip_other and label == "O":
                    continue
                token_types.update([
                    label
                ])
        except IndexError:
            print(i, seq)
            raise
    return total_seq, total_tokens, token_types
        

In [6]:
def make_conll_dataset_tables(files, **kwargs):
    all_stats = []
    for datakey in files:
        for datatype, filepath in files[datakey].items():
            print("{}-{}: {}".format(datakey, datatype, filepath))
            # replace datadrive path with current data drive
            filepath = filepath.replace("/datadrive/", "")
            filepath = os.path.join(DATA_DRIVE_PATH, filepath)
            total_seq, total_tokens, token_types = get_file_stats(filepath, **kwargs)
            print(total_seq, total_tokens, token_types)
            all_stats.append((datakey, datatype, total_seq, total_tokens, token_types))
    return all_stats


def generate_tables(files, display_df=False, show_labels=True, **kwargs):
    all_stats = make_conll_dataset_tables(files, **kwargs)
    df = pd.DataFrame(all_stats, columns=[
        "datakey", "datatype", "total_seq", "total_tokens", "labels"])
    if show_labels:
        df = df.assign(
            all_labels=df["labels"].apply(lambda x: (", ".join(sorted(x.keys()))).upper())
        )
    df = df.assign(
        num_labels=df["labels"].apply(len),
    ).sort_values(["datakey", "datatype"])
    if display_df:
        display(df)
    with pd.option_context("display.max_colwidth", -1):
        print(df.drop("labels", 1).set_index(["datakey", "datatype"]).to_latex())
        display(df.drop("labels", 1).set_index(["datakey", "datatype"]))
    

In [7]:
generate_tables(NER_FILES, display_df=True, label_processor=lambda x: get_ner_label(x, idx=1))

Finin-train: /datadrive/Datasets/lowlands-data/LREC2014/twitter_ner/data/finin.train.tsv
10000 172188 Counter({'PER': 6675, 'ORG': 2975, 'LOC': 2360})
Finin-test: /datadrive/Datasets/lowlands-data/LREC2014/twitter_ner/data/finin.test.tsv.utf8
2975 51056 Counter({'PER': 1563, 'ORG': 595, 'LOC': 544})
Hege-test: /datadrive/Datasets/lowlands-data/LREC2014/twitter_ner/data/hege.test.tsv
1545 20664 Counter({'PER': 1677, 'ORG': 284, 'LOC': 215})
Ritter-lowlands-test: /datadrive/Datasets/lowlands-data/LREC2014/twitter_ner/data/ritter.test.tsv
2394 46469 Counter({'PER': 656, 'LOC': 533, 'ORG': 392})
Ritter-train: /datadrive/Datasets/Twitter/RitterNER/twitter_processed/ner.train.txt
1900 36936 Counter({'PERSON': 521, 'OTHER': 449, 'GEO-LOC': 254, 'COMPANY': 179, 'FACILITY': 170, 'PRODUCT': 141, 'MUSICARTIST': 91, 'MOVIE': 65, 'SPORTSTEAM': 59, 'TVSHOW': 45})
Ritter-dev: /datadrive/Datasets/Twitter/RitterNER/twitter_processed/ner.dev.txt
240 4612 Counter({'PERSON': 79, 'OTHER': 52, 'PRODUCT': 31

,datakey,datatype,total_seq,total_tokens,labels,all_labels,num_labels
20,BROAD,a,1000,16612,"{'LOC': 168, 'PER': 331, 'ORG': 251}","LOC, ORG, PER",3
21,BROAD,b,2000,30825,"{'PER': 1079, 'ORG': 667, 'LOC': 310}","LOC, ORG, PER",3
22,BROAD,e,200,3981,"{'ORG': 184, 'LOC': 182, 'PER': 74}","LOC, ORG, PER",3
23,BROAD,f,2001,35428,"{'PER': 2919, 'LOC': 844, 'ORG': 1336}","LOC, ORG, PER",3
24,BROAD,g,2138,34062,"{'ORG': 2010, 'PER': 2555, 'LOC': 1853}","LOC, ORG, PER",3
25,BROAD,h,2001,29480,"{'PER': 2524, 'ORG': 863, 'LOC': 423}","LOC, ORG, PER",3
27,BROAD,split-dev,933,15169,"{'PER': 987, 'LOC': 376, 'ORG': 574}","LOC, ORG, PER",3
28,BROAD,split-test,2802,45159,"{'ORG': 1596, 'PER': 2768, 'LOC': 1080}","LOC, ORG, PER",3
26,BROAD,split-train,5605,90060,"{'PER': 5727, 'ORG': 3141, 'LOC': 2324}","LOC, ORG, PER",3
1,Finin,test,2975,51056,"{'LOC': 544, 'ORG': 595, 'PER': 1563}","LOC, ORG, PER",3


\begin{tabular}{llrrlr}
\toprule
      &       &  total\_seq &  total\_tokens &                                                                                                                                                                                                           all\_labels &  num\_labels \\
datakey & datatype &            &               &                                                                                                                                                                                                                      &             \\
\midrule
BROAD & a &  1000 &  16612 &  LOC, ORG, PER &  3 \\
      & b &  2000 &  30825 &  LOC, ORG, PER &  3 \\
      & e &  200 &  3981 &  LOC, ORG, PER &  3 \\
      & f &  2001 &  35428 &  LOC, ORG, PER &  3 \\
      & g &  2138 &  34062 &  LOC, ORG, PER &  3 \\
      & h &  2001 &  29480 &  LOC, ORG, PER &  3 \\
      & split-dev &  933 &  15169 &  LOC, ORG, PER &  3 \\
      & split-test &  2802 &  

total_seq  total_tokens  \
datakey    datatype                                 
BROAD      a              1000       16612          
           b              2000       30825          
           e              200        3981           
           f              2001       35428          
           g              2138       34062          
           h              2001       29480          
           split-dev      933        15169          
           split-test     2802       45159          
           split-train    5605       90060          
Finin      test           2975       51056          
           train          10000      172188         
Hege       test           1545       20664          
MSM_2013   test           1450       29089          
           train          2815       51521          
MultiModal dev            1000       16178          
           test           3257       52822          
           train          4000       64439          
NEEL2016   dev            88         1647           
           test           2663       47488          
           train          2588       51669          
Ritter     dev            240        4612           
           lowlands-test  2394       46469          
           test           254        4921           
           train          1900       36936          
WNUT_2016  dev            1000       16261          
           test           3850       61908          
           train          2394       46469          
WNUT_2017  dev            1009       15733          
           test           1287       23394          
           train          3394       62730          
YODIE      test           397        8032           
           train          396        7905           

                                                                                                                                                                                                                                   all_labels  \
datakey    datatype                                                                                                                                                                                                                             
BROAD      a              LOC, ORG, PER                                                                                                                                                                                                         
           b              LOC, ORG, PER                                                                                                                                                                                                         
           e              LOC, ORG, PER                                                                                                                                                                                                         
           f              LOC, ORG, PER                                                                                                                                                                                                         
           g              LOC, ORG, PER                                                                                                                                                                                                         
           h              LOC, ORG, PER                                                                                                                                                                                                         
           split-dev      LOC, ORG, PER                                                                                                                                                                                                         
           split-test     LOC, ORG, PER               

## POS datasets

In [8]:
generate_tables(POS_FILES, display_df=False)

Owoputi_2013-train: /datadrive/Datasets/Twitter/TweeboParser/ark-tweet-nlp-0.3.2/data/twpos-data-v0.3/oct27.splits/oct27.train
1000 14619 Counter({'V': 2219, 'N': 2003, ',': 1715, 'P': 1252, '^': 890, 'D': 869, 'A': 755, '@': 713, 'R': 689, '~': 538, '!': 406, 'L': 252, '&': 239, 'U': 223, '$': 216, 'E': 148, '#': 141, 'G': 137, 'T': 92, 'Z': 21, 'S': 18, 'X': 15, 'M': 3, 'Y': 2})
Owoputi_2013-traindev: /datadrive/Datasets/Twitter/TweeboParser/ark-tweet-nlp-0.3.2/data/twpos-data-v0.3/oct27.splits/oct27.traindev
1327 19442 Counter({'V': 2970, 'N': 2663, ',': 2215, 'P': 1692, '^': 1201, 'D': 1181, 'A': 994, '@': 956, 'R': 898, '~': 708, '!': 505, '&': 330, 'L': 317, 'U': 314, '$': 302, 'G': 202, 'E': 200, '#': 193, 'T': 128, 'Z': 30, 'S': 23, 'X': 19, 'M': 3, 'Y': 2})
Owoputi_2013-dev: /datadrive/Datasets/Twitter/TweeboParser/ark-tweet-nlp-0.3.2/data/twpos-data-v0.3/oct27.splits/oct27.dev
327 4823 Counter({'V': 751, 'N': 660, ',': 500, 'P': 440, 'D': 312, '^': 311, '@': 243, 'A': 239, 'R

total_seq  total_tokens  \
datakey      datatype                                 
Foster_2011  test           250        2841           
             twitie-dev     269        2998           
             twitie-test    250        2841           
Gimple_2012  test           1000       14439          
Owoputi_2013 daily547       547        7707           
             dev            327        4823           
             test           500        7152           
             train          1000       14619          
             traindev       1327       19442          
Ritter_2011  dev            71         1362           
             lowlands-test  118        2291           
             test           84         1627           
             train          632        12196          
Tweetbankv2  dev            710        11759          
             test           1201       19095          
             train          1639       24753          
lowlands     test           200        3064           

                                                                                                                                                                                                                  all_labels  \
datakey      datatype                                                                                                                                                                                                          
Foster_2011  test           ., ADJ, ADP, ADV, CONJ, DET, NOUN, NUM, PRON, PRT, VERB, X                                                                                                                                         
             twitie-dev     '', (, ), ,, ., :, CC, CD, DT, FW, HT, IN, JJ, JJR, JJS, MD, NN, NNP, NNPS, NNS, PDT, POS, PRP, PRP$, RB, RBR, RBS, RP, RT, SYM, TO, UH, URL, USR, VB, VBD, VBG, VBN, VBP, VBZ, WDT, WP, WRB       
             twitie-test    '', (, ), ,, ., :, CC, CD, DT, EX, FW, HT, IN, JJ, JJR, JJS, MD, NN, NNP, NNPS, NNS, PDT, POS, PRP, PRP$, RB, RBR, RBS, RP, RT, SYM, TO, UH, URL, USR, VB, VBD, VBG, VBN, VBP, VBZ, WDT, WP, WRB   
Gimple_2012  test           ., ADJ, ADP, ADV, CONJ, DET, NOUN, NUM, PRON, PRT, VERB, X                                                                                                                                         
Owoputi_2013 daily547       !, #, $, &, ,, @, A, D, E, G, L, M, N, P, R, S, T, U, V, X, Y, Z, ^, ~                                                                                                                             
             dev            !, #, $, &, ,, @, A, D, E, G, L, N, P, R, S, T, U, V, X, Z, ^, ~                                                                                                                                   
             test           !, #, $, &, ,, @, A, D, E, G, L, N, P, R, S, T, U, V, X, Z, ^, ~                                                                                                                                   
             train          !, #, $, &, ,, @, A, D, E, G, L, M, N, P, R, S, T, U, V, X, Y, Z, ^, ~                                                                                                                             
             traindev       !, #, $, &, ,, @, A, D, E, G, L, M, N, P, R, S, T, U, V, X, Y, Z, ^, ~                                                                                                                             
Ritter_2011  dev            '', (, ), ,, ., :, CC, CD, DT, HT, IN, JJ, JJR, JJS, MD, NN, NNP, NNS, POS, PRP, PRP$, RB, RBR, RP, RT, TO, UH, URL, USR, VB, VBD, VBG, VBN, VBP, VBZ, WDT, WP, WRB                                
             lowlands-test  ., ADJ, ADP, ADV, CONJ, DET, NOUN, NUM, PRON, PRT, VERB, X                                                                                                                                         
             test           '', (, ), ,, ., :, CC, CD, DT, EX, HT, IN,

## Chunking

In [9]:
generate_tables(CHUNKING_FILES, label_processor=lambda x: get_ner_label(x, idx=1))

Ritter-train: /datadrive/Datasets/Twitter/RitterNER/twitter_processed/chunk.train.conll
551 10584 Counter({'NP': 4681, 'VP': 1901, 'PP': 748, 'ADVP': 403, 'INTJ': 355, 'ADJP': 211, 'PRT': 75, 'SBAR': 73, 'CONJP': 4})
Ritter-dev: /datadrive/Datasets/Twitter/RitterNER/twitter_processed/chunk.dev.conll
118 2317 Counter({'NP': 1020, 'VP': 397, 'PP': 148, 'ADVP': 107, 'INTJ': 71, 'ADJP': 50, 'PRT': 22, 'SBAR': 10})
Ritter-test: /datadrive/Datasets/Twitter/RitterNER/twitter_processed/chunk.test.conll
119 2310 Counter({'NP': 977, 'VP': 428, 'PP': 141, 'ADVP': 91, 'INTJ': 82, 'ADJP': 66, 'SBAR': 25, 'PRT': 12})
\begin{tabular}{llrrlr}
\toprule
       &       &  total\_seq &  total\_tokens &                                      all\_labels &  num\_labels \\
datakey & datatype &            &               &                                                 &             \\
\midrule
Ritter & dev &  118 &  2317 &  ADJP, ADVP, INTJ, NP, PP, PRT, SBAR, VP &  8 \\
       & test &  119 &  2310 &  ADJP, 

total_seq  total_tokens  \
datakey datatype                            
Ritter  dev       118        2317           
        test      119        2310           
        train     551        10584          

                                                      all_labels  num_labels  
datakey datatype                                                              
Ritter  dev       ADJP, ADVP, INTJ, NP, PP, PRT, SBAR, VP         8           
        test      ADJP, ADVP, INTJ, NP, PP, PRT, SBAR, VP         8           
        train     ADJP, ADVP, CONJP, INTJ, NP, PP, PRT, SBAR, VP  9

## Supersense tagging

In [10]:
generate_tables(SUPERSENSE_TAGGING_FILES, display_df=False)

Ritter-train: /datadrive/Datasets/Twitter/supersense-data-twitter/ritter-train.tsv
551 10652 Counter({'B-noun.time': 520, 'B-verb.stative': 399, 'B-noun.person': 260, 'B-noun.communication': 224, 'B-noun.artifact': 167, 'B-verb.social': 161, 'B-verb.communication': 151, 'B-verb.motion': 136, 'B-verb.cognition': 130, 'B-noun.location': 127, 'B-verb.possession': 119, 'B-noun.act': 113, 'B-noun.group': 104, 'B-verb.change': 103, 'I-noun.communication': 102, 'B-verb.emotion': 94, 'I-noun.person': 85, 'B-noun.event': 69, 'I-noun.artifact': 62, 'B-verb.perception': 61, 'I-noun.time': 55, 'I-noun.event': 53, 'I-noun.act': 47, 'I-noun.group': 47, 'B-noun.state': 41, 'I-noun.location': 41, 'B-verb.contact': 39, 'B-noun.cognition': 37, 'B-noun.food': 35, 'B-noun.attribute': 34, 'B-verb.creation': 32, 'B-noun.quantity': 31, 'B-verb.body': 27, 'B-noun.body': 26, 'B-verb.competition': 23, 'B-noun.possession': 17, 'B-noun.animal': 16, 'B-noun.relation': 15, 'B-verb.consumption': 12, 'B-noun.substanc

total_seq  total_tokens  \
datakey        datatype                            
Johannsen_2014 test      200        3064           
Ritter         dev       118        2242           
               test      118        2291           
               train     551        10652          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   all_labels  \
datakey        datatype                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
Johannsen_2014 test      B-NOUN.ACT, B-NOUN.ANIMAL, B-NOUN.ARTIFACT, B-NOUN.ATTRIBUTE, B-NOUN.BODY, B-NOUN.COGNITION, B-NOUN.COMMUNICATION, B-NOUN.EVENT, B-NOUN.FEELING, B-NOUN.FOOD, B-NOUN.GROUP, B-NOUN.LOCATION, B-NOUN.OBJECT, B-NOUN.PERSON, B-NOUN.PHENOMENON, B-NOUN.POSSESSION, B-NOUN.PROCESS, B-NOUN.QUANTITY, B-NOUN.RELATION, B-NOUN.SHAPE, B-NOUN.STATE, B-NOUN.SUBSTANCE, B-NOUN.TIME, B-VERB.BODY, B-VERB.CHANGE, B-VERB.COGNITION, B-VERB.COMMUNICATION, B-VERB.COMPETITION, B-VERB.CONSUMPTION, B-VERB.CONTACT, B-VERB.CREATION, B-VERB.EMOTION, B-VERB.MOTION, B-VERB.PERCEPTION, B-VERB.POSSESSION, B-VERB.SOCIAL, B-VERB.STATIVE, I-NOUN.ACT, I-NOUN.ANIMAL, I-NOUN.ARTIFACT, I-NOUN.ATTRIBUTE, I-NOUN.COGNITION, I-NOUN.COMMUNICATION, I-NOUN.EVENT, I-NOUN.FOOD, I-NOUN.GROUP, I-NOUN.LOCATION, I-NOUN.PERSON, I-NOUN.POSSESSION, I-NOUN.QUANTITY, I-NOUN.STATE, I-NOUN.TIME, I-VERB.SOCIAL                                                                                                                                                                                                             
Ritter         dev       B-NOUN.ACT, B-NOUN.ANIMAL, B-NOUN.ARTIFACT, B-NOUN.ATTRIBUTE, B-NOUN.BODY, B-NOUN.COGNITION, B-NOUN.COMMUNICATION, B-NOUN.EVENT, B-NOUN.FEELING, B-NOUN.FOOD, B-NOUN.GROUP, B-NOUN.LOCATION, B-NOUN.MOTIVE, B-NOUN.OBJECT, B-NOUN.PERSON, B-NOUN.PHENOMENON, B-NOUN.PLANT, B-NOUN.POSSESSION, B-NOUN.QUANTITY, B-NOUN.RELATION, B-NOUN.STATE, B-NOUN.SUBSTANCE, B-NOUN.TIME, B-VERB.BODY, B-VERB.CHANGE, B-VERB.COGNITION, B-VERB.CO

In [11]:
generate_tables(SUPERSENSE_TAGGING_FILES, label_processor=lambda x: get_ner_label(x, idx=1))

Ritter-train: /datadrive/Datasets/Twitter/supersense-data-twitter/ritter-train.tsv
551 10652 Counter({'NOUN.TIME': 575, 'VERB.STATIVE': 400, 'NOUN.PERSON': 345, 'NOUN.COMMUNICATION': 326, 'NOUN.ARTIFACT': 229, 'NOUN.LOCATION': 168, 'VERB.SOCIAL': 162, 'NOUN.ACT': 160, 'VERB.COMMUNICATION': 152, 'NOUN.GROUP': 151, 'VERB.MOTION': 136, 'VERB.COGNITION': 131, 'NOUN.EVENT': 122, 'VERB.POSSESSION': 120, 'VERB.CHANGE': 103, 'VERB.EMOTION': 94, 'VERB.PERCEPTION': 62, 'NOUN.COGNITION': 46, 'NOUN.FOOD': 45, 'NOUN.STATE': 42, 'VERB.CONTACT': 39, 'NOUN.ATTRIBUTE': 38, 'VERB.CREATION': 32, 'NOUN.QUANTITY': 32, 'VERB.BODY': 27, 'NOUN.BODY': 27, 'NOUN.POSSESSION': 26, 'VERB.COMPETITION': 23, 'NOUN.ANIMAL': 18, 'NOUN.RELATION': 16, 'NOUN.PHENOMENON': 12, 'VERB.CONSUMPTION': 12, 'NOUN.SUBSTANCE': 12, 'NOUN.TOPS': 11, 'NOUN.FEELING': 10, 'NOUN.OBJECT': 6, 'NOUN.PROCESS': 3, 'NOUN.MOTIVE': 3, 'NOUN.PLANT': 2, 'VERB.WEATHER': 1})
Ritter-dev: /datadrive/Datasets/Twitter/supersense-data-twitter/ritter-dev.t

total_seq  total_tokens  \
datakey        datatype                            
Johannsen_2014 test      200        3064           
Ritter         dev       118        2242           
               test      118        2291           
               train     551        10652          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        all_labels  \
datakey        datatype                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
Johannsen_2014 test      NOUN.ACT, NOUN.ANIMAL, NOUN.ARTIFACT, NOUN.ATTRIBUTE, NOUN.BODY, NOUN.COGNITION, NOUN.COMMUNICATION, NOUN.EVENT, NOUN.FEELING, NOUN.FOOD, NOUN.GROUP, NOUN.LOCATION, NOUN.OBJECT, NOUN.PERSON, NOUN.PHENOMENON, NOUN.POSSESSION, NOUN.PROCESS, NOUN.QUANTITY, NOUN.RELATION, NOUN.SHAPE, NOUN.STATE, NOUN.SUBSTANCE, NOUN.TIME, VERB.BODY, VERB.CHANGE, VERB.COGNITION, VERB.COMMUNICATION, VERB.COMPETITION, VERB.CONSUMPTION, VERB.CONTACT, VERB.CREATION, VERB.EMOTION, VERB.MOTION, VERB.PERCEPTION, VERB.POSSESSION, VERB.SOCIAL, VERB.STATIVE                                         
Ritter         dev       NOUN.ACT, NOUN.ANIMAL, NOUN.ARTIFACT, NOUN.ATTRIBUTE, NOUN.BODY, NOUN.COGNITION, NOUN.COMMUNICATION, NOUN.EVENT, NOUN.FEELING, NOUN.FOOD, NOUN.GROUP, NOUN.LOCATION, NOUN.MOTIVE, NOUN.OBJECT, NOUN.PERSON, NOUN.PHENOMENON, NOUN.PLANT, NOUN.POSSESSION, NOUN.QUANTITY, NOUN.RELATION, NOUN.STATE, NOUN.SUBSTANCE, NOUN.TIME, VERB.BODY, VERB.CHANGE, VERB.COGNITION, VERB.COMMUNICATION, VERB.COMPETITION, VERB.CONSUMPTION, VERB.CONTACT, VERB.CREATION, VERB.EMOTION, VERB.MOTION, VERB.PERCEPTION, VERB.POSSESSION, VERB.SOCIAL, VERB.STATIVE                                          
               test      NOUN.ACT, NOUN.ANIMAL, NOUN.ARTIFACT, NOUN.ATTRIBUTE, NOUN.BODY, NOUN.COGNITION, NOUN.COMMUNICATION, NOUN.EVENT, NOUN.FEELING, NOUN.FOOD, NOUN.GROUP, NOUN.LOCATION, NOUN.MOTIVE, NOUN.OBJECT, NOUN.PERSON, NOUN.PHENOMENON, NOUN.POSSESSION, NOUN.PROCESS, NOUN.QUANTITY, NOUN.RELATION, NOUN.SHAPE, NOUN.STATE, NOUN.SUBSTANCE, NOUN.TIME, NOUN.TOPS, VERB.BODY, VERB.CHANGE, VERB.COGNITION, VERB.COMMUNICATION, VERB.COMPETITION, VERB.CONSUMPTION, VERB.CONTACT, VERB.CREATION, VERB.EMOTION, VERB.MOTION, VERB.PERCEPTION, VERB.POSSESSION, VERB.SOCIAL, VERB.STATIVE, VERB.WEATHER   
               train     NOUN.ACT, NOUN.ANIMAL, NOUN.ARTIFACT, NOUN.ATTRIBUTE, NOUN.BODY, NOUN.COGNITION, NOUN.COMMUNICATION, NOUN.EVENT, NOUN.FEELING, NOUN.FOOD, NOUN.GROUP, NOUN.LOCATION, NOUN.MOTIVE, NOUN.OBJECT, NOUN.PERSON, NOUN.PHENOMENON, NOUN.PLANT, NOUN.POSSESSION, NOUN.PROCESS, NOUN.QUANTITY, NOUN.RELATION, NOUN.STATE, NOUN.SUBSTANCE, NOUN.TIME, NOUN.TOPS, VERB.BODY, VERB.CHANGE, VERB.COGNITION, VERB.COMMUNICATION, VERB.COMPETITION, VERB.CONSUMPTION, VERB.CONTACT, VERB.CREATION, VERB.EMOTION, VERB.MOTION, VERB.PERCEPTION, VERB.POSSESSION, VERB.SOCIAL, VERB.STATIVE, VERB.WEATHER   

                         num_labels  
datakey        datatype              
Johannsen_2014 test      37          
Ritter   

## DimSUM 

https://dimsum16.github.io/

In [12]:
generate_tables(DIMSUM_FILES, label_col_id=7, label_processor=get_simple_label, skip_other=True)

DiMSUM_2016-train: /datadrive/Datasets/Twitter/dimsum-data/conll/dimsum16.train
4799 73826 Counter({'v.stative': 3357, 'n.person': 1867, 'n.group': 1750, 'v.cognition': 1310, 'n.artifact': 1226, 'v.communication': 1225, 'n.time': 1173, 'v.social': 1073, 'n.act': 902, 'n.communication': 886, 'n.location': 845, 'n.cognition': 835, 'n.food': 830, 'v.motion': 765, 'n.event': 650, 'v.change': 539, 'v.emotion': 414, 'v.possession': 398, 'n.possession': 386, 'n.attribute': 311, 'v.perception': 241, 'n.quantity': 161, 'n.body': 153, 'v.body': 153, 'n.state': 122, 'n.animal': 122, 'v.creation': 117, 'v.consumption': 114, 'v.contact': 104, 'n.feeling': 68, 'n.natural_object': 62, 'n.substance': 52, 'v.competition': 51, 'n.phenomenon': 46, 'n.relation': 40, 'n.motive': 35, 'n.process': 29, 'n.other': 23, 'n.plant': 10, 'n.shape': 7, 'v.weather': 2})
DiMSUM_2016-test: /datadrive/Datasets/Twitter/dimsum-data/conll/dimsum16.test
1000 16500 Counter({'v.stative': 769, 'n.person': 413, 'n.artifact': 33

total_seq  total_tokens  \
datakey     datatype                            
DiMSUM_2016 test      1000       16500          
            train     4799       73826          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               all_labels  \
datakey     datatype                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
DiMSUM_2016 test      N.ACT, N.ANIMAL, N.ARTIFACT, N.ATTRIBUTE, N.BODY, N.COGNITION, N.COMMUNICATION, N.EVENT, N.FEELING, N.FOOD, N.GROUP, N.LOCATION, N.MOTIVE, N.NATURAL_OBJECT, N.OTHER, N.PERSON, N.PHENOMENON, N.PLANT, N.POSSESSION, N.PROCESS, N.QUANTITY, N.RELATION, N.SHAPE, N.STATE, N.SUBSTANCE, N.TIME, V.BODY, V.CHANGE, V.COGNITION, V.COMMUNICATION, V.COMPETITION, V.CONSUMPTION, V.CONTACT, V.CREATION, V.EMOTION, V.MOTION, V.PERCEPTION, V.POSSESSION, V.SOCIAL, V.STATIVE, V.WEATHER   
            train     N.ACT, N.ANIMAL, N.ARTIFACT, N.ATTRIBUTE, N.BODY, N.COGNITION, N.COMMUNICATION, N.EVENT, N.FEELING, N.FOOD, N.GROUP, N.LOCATION, N.MOTIVE, N.NATURAL_OBJECT, N.OTHER, N.PERSON, N.PHENOMENON, N.PLANT, N.POSSESSION, N.PROCESS, N.QUANTITY, N.RELATION, N.SHAPE, N.STATE, N.SUBSTANCE, N.TIME, V.BODY, V.CHANGE, V.COGNITION, V.COMMUNICATION, V.COMPETITION, V.CONSUMPTION, V.CONTACT, V.CREATION, V.EMOTION, V.MOTION, V.PERCEPTION, V.POSSESSION, V.SOCIAL, V.STATIVE, V.WEATHER   

                      num_labels  
datakey     datatype              
DiMSUM_2016 test      41          
            train     41

## Frame Semantics



```
@paper{AAAI159349,
	author = {Anders Søgaard and Barbara Plank and Hector Alonso},
	title = {Using Frame Semantics for Knowledge Extraction from Twitter},
	conference = {AAAI Conference on Artificial Intelligence},
	year = {2015},
	keywords = {frame semantics; knowledge bases; twitter},
	abstract = {Knowledge bases have the potential to advance artificial intelligence, but often suffer from recall problems, i.e., lack of knowledge of new entities and relations. On the contrary, social media such as Twitter provide abundance of data, in a timely manner: information spreads at an incredible pace and is posted long before it makes it into more commonly used resources for knowledge extraction. In this paper we address the question whether we can exploit social media to extract new facts, which may at first seem like finding needles in haystacks. We collect tweets about 60 entities in Freebase and compare four methods to extract binary relation candidates, based on syntactic and semantic parsing and simple mechanism for factuality scoring. The extracted facts are manually evaluated in terms of their correctness and relevance for search. We show that moving from bottom-up syntactic or semantic dependency parsing formalisms to top-down frame-semantic processing improves the robustness of knowledge extraction, producing more intelligible fact candidates of better quality. In order to evaluate the quality of frame semantic parsing on Twitter intrinsically, we make a multiply frame-annotated dataset of tweets publicly available.},

	url = {https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/view/9349}
}

```

In [13]:
generate_tables(FRAME_SEMANTICS_FILE, show_labels=False, label_col_id=3, label_processor=get_simple_label, skip_other=True)

Sogaard_2015-gavin: /datadrive/Datasets/lowlands-data/AAAI15/conll/all.gavin
236 4533 Counter({'Calendric_unit': 106, '{}': 33, 'Intentionally_act': 22, 'Measure_duration': 21, 'Have_associated': 21, 'Arriving': 20, 'Temporal_collocation': 19, 'Desirability': 19, 'Desiring': 18, 'Getting': 17, 'Motion': 17, 'Relative_time': 16, 'Quantity': 15, 'Observable_body_parts': 15, 'Locale_by_use': 13, 'Kinship': 12, 'Event_instance': 12, 'Becoming': 12, 'Experiencer_focus': 11, 'Age': 9, 'People': 9, 'Opinion': 8, 'Opportunity': 8, 'Social_event': 8, 'Capability': 8, 'Text': 8, 'Food': 8, 'Likelihood': 8, 'Perception_active': 8, 'Competition': 7, 'Performers_and_roles': 7, 'Make_noise': 7, 'Sounds': 7, 'Ordinal_numbers': 7, 'Personal_relationship': 7, 'Expensiveness': 7, 'Part_orientational': 7, 'Color': 7, 'Cotheme': 6, 'Location_in_time': 6, 'Contacting': 6, 'Communication': 6, 'Ingestion': 6, 'Political_locales': 6, 'Taking': 5, 'Hit_or_miss': 5, 'Size': 5, 'Emotion_directed': 5, 'Temporal_s

total_seq  total_tokens  num_labels
datakey      datatype                                     
Sogaard_2015 gavin     236        4533          310       
             maria     236        4533          306       
             sara      236        4533          306